# top

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import json
import wandb
import torch.nn as nn
from torch.utils.data import DataLoader

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
print(sys.path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

['/mnt/ceph/users/blyo1/projects/hdiva', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python310.zip', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/lib-dynload', '', '/mnt/home/blyo1/venvs/py310/lib/python3.10/site-packages']


# load checkpoint

In [ ]:
from c_training.configs.diva_config import DiVA_Manual_CelebA_64_Training_Config
from utils.wandb_utils import get_wandb_run_name, get_artifact

project_name = "diva_manual_celeba_color_64"
model_num = 71

# construct config from wandb run
model_path, artifact, config, run_config = get_artifact(model_num=model_num, project_name=project_name)
config = DiVA_Manual_CelebA_64_Training_Config.from_dict(run_config)

wandb run name: ckv0t0st


wandb: Downloading large artifact model-ckv0t0st:latest, 477.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.8


In [20]:
from c_training.diva_lightning import DiVA_Lightning

lightning_model = DiVA_Lightning.load_from_checkpoint(checkpoint_path=model_path, config=config)


# test the model
from b_models.diva.diva_inference import DiVA_Inference
diva_inference = DiVA_Inference(diva=lightning_model.model, config=config)
diva_inference.to(device);
# diva = lightning_model.diva.to(device)

In [ ]:
lightning_model.optimizer.load_state_dict()

In [27]:
from lightning.pytorch import Trainer

base_dir = "/mnt/home/blyo1/hdiva"
trainer = Trainer(
    accelerator="cuda",
    devices=1,  # GPUs per node (adjust based on your setup)
    num_nodes=1,  # Number of nodes
    max_epochs=config.num_epochs,
    log_every_n_steps=config.log_every_n_steps,
    precision=config.precision,
    enable_checkpointing=True,
    enable_progress_bar=True,
    default_root_dir=f"{base_dir}",
)

model = DiVA_Lightning(config=config)
trainer.fit(model, ckpt_path=model_path)

/mnt/home/blyo1/venvs/py310/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/home/blyo1/venvs/py310/lib/python3.10/site-pack ...
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /mnt

KeyError: 'Trying to restore optimizer state but checkpoint contains only the model. This is probably due to `ModelCheckpoint.save_weights_only` being set to `True`.'